In [1]:
!pip install langchain
!pip install sentence-transformers
!pip install faiss-cpu
!pip install bs4
!pip install replicate
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB

In [6]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [2]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN


··········


In [11]:
from langchain_community.llms import Replicate
llm = Replicate(
    model="meta/meta-llama-3-8b-instruct",
    model_kwargs={"temperature": 0.0, "top_p": 1, "max_new_tokens":500}
)

In [12]:
question = "who wrote the book Innovator's dilemma?"
answer = llm.invoke(question)
print(answer)

The book "The Innovator's Dilemma: When New Technologies Cause Great Firms to Fail" was written by Clayton M. Christensen, a Harvard Business School professor. The book was first published in 1997 and has since become a classic in the field of business and innovation.


In [ ]:

followup = "tell me more"
followup_answer = llm.invoke(followup)
print(followup_answer)

I'd be happy to tell you more about myself.

I'm an AI assistant, which means I'm a computer program designed to assist and communicate with humans. My primary goal is to provide helpful and accurate information to answer your questions and complete tasks for you.

Here are some things I can do:

1. **Answer questions**: I can process natural language queries and provide relevant answers. Whether you're looking for general knowledge, definitions, or specific information, I'm here to help.
2. **Generate text**: I can create text based on a prompt or topic. This can be useful for writing articles, emails, or even entire books.
3. **Translate languages**: I can translate text from one language to another. I currently support translations in dozens of languages.
4. **Summarize content**: If you have a long piece of text and want to get a quick summary of the main points, I can help with that.
5. **Offer suggestions**: I can provide suggestions for things like gift ideas, travel destination

In [14]:

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [ ]:

answer = conversation.predict(input=question)
print(answer)

The book "The Innovator's Dilemma: When New Technologies Cause Great Firms to Fail" was written by Clayton Christensen, an American academic and business theorist. He's a professor at Harvard Business School and is known for his work on innovation, disruption, and strategy. The book was first published in 1997 and has since become a classic in the field of business and management. It explores the idea that established companies can struggle to innovate and adapt to new technologies, leading to their downfall.


In [ ]:
# pass context (previous question and answer) along with the follow up "tell me more" to Llama who now knows more of what
memory.save_context({"input": question},
                    {"output": answer})
followup_answer = conversation.predict(input=followup)
print(followup_answer)

I'd be happy to tell you more about the book and its author! Clayton Christensen's work is highly regarded in the business and academic communities, and "The Innovator's Dilemma" is considered one of his most influential books.

In the book, Christensen argues that established companies often struggle to innovate and adapt to new technologies because of their existing business models, organizational structures, and cultural norms. He uses the term "disruptive innovation" to describe the process by which new technologies or business models disrupt the status quo and create new markets or opportunities.

Christensen also introduces the concept of the "innovator's dilemma," which refers to the tension that exists between the need to innovate and the need to maintain profitability. He argues that established companies often prioritize short-term financial performance over long-term innovation, which can lead to their downfall.

The book is filled with examples of companies that have succes

In [15]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(["https://huggingface.co/blog/llama3"])
docs = loader.load()


In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(all_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever()
)

question = "What's new with Llama 3?"
result = qa_chain({"query": question})
print(result['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


According to the provided context, the new features and changes in Llama 3 compared to Llama 2 are:

* A new tokenizer that expands the vocabulary size to 128,256 (from 32K tokens in the previous version)
* The introduction of 4 new open LLM models based on the Llama 2 architecture, in two sizes: 8B and 70B parameters, each with base (pre-trained) and instruct-tuned versions
* The ability to run on various types of consumer hardware and have a context length of 8K tokens
* A new version of Llama Guard, fine-tuned on Llama 3 8B and released as Llama Guard 2 (safety fine-tune)

These changes aim to improve the efficiency and multilingualism of the models, while also providing more deployment options and a larger vocabulary size.


In [ ]:

result = qa_chain({"query": "Based on what architecture?"})
print(result['result'])

Based on the provided context, it appears that Llama 3 is based on the Meta Llama architecture.


In [ ]:

from langchain.chains import ConversationalRetrievalChain
chat_chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [ ]:

result = chat_chain({"question": question, "chat_history": []})
print(result['answer'])

According to the provided context, the new features and changes in Llama 3 compared to Llama 2 are:

* A new tokenizer that expands the vocabulary size to 128,256 (from 32K tokens in the previous version)
* The introduction of 4 new open LLM models based on the Llama 2 architecture, in two sizes: 8B and 70B parameters, each with base (pre-trained) and instruct-tuned versions
* The ability to run on various types of consumer hardware and have a context length of 8K tokens
* A new version of Llama Guard, fine-tuned on Llama 3 8B and released as Llama Guard 2 (safety fine-tune)

These changes aim to improve the efficiency and multilingualism of the models, while also providing more deployment options and a larger vocabulary size.


In [ ]:

chat_history = [(question, result["answer"])]
followup = "Based on what architecture?"
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

Based on the provided context, it appears that Llama 3 is based on the Meta Llama architecture.


In [ ]:

chat_history.append((followup, followup_answer["answer"]))
more_followup = "What changes in vocabulary size?"
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])

According to the text, the vocabulary size has increased from 32K tokens in Llama 2 to 128,256 tokens in Llama 3.
